# Getting viral sequences from virome sequencing data
Anneliek ter Horst, March 2023

We are starting with sequencing data from UC Davis Russell Ranch Sustainable Agriculture Facility, Davis, CA. Part of this is done using Snakemake

1. Clean sequencing reads: remove adapter sequences using trimmomatic
2. Clean sequencing reads: remove PhiX spike using bbduk
3. Assembly into contigs using MEGAHIT

## Main snakefile:
Start with a text file thats a list of samples

In [ ]:
# main snakefile:

# include all the other snakefiles
include: "trimmomatic.snakefile"

# import pandas
import pandas as pd

# Import sample names
samples_df =  pd.read_csv('samples.txt', sep='\t').set_index("sample", drop=False)
SAMPLES = list(samples_df['sample'])


# input is dus actually the final output
rule all:
    input:
         expand("reads/trimmomatic/out/{sample}_trim_done.txt", sample=SAMPLES)



## Trimmomatic
- Trim adapter sequences
- http://www.usadellab.org/cms/?page=trimmomatic
- Bolger, A. M., Lohse, M., & Usadel, B. (2014). Trimmomatic: A flexible trimmer for Illumina Sequence Data. Bioinformatics, btu170.

In [ ]:
# trimmomatic snakefile
# run trimmomatic on raw reads
rule trimmomatic:
    input:
        forward_raw = "reads/raw/{sample}_R1.fastq.gz",
        reverse_raw = "reads/raw/{sample}_R2.fastq.gz"
    output:
        forward_paired = "reads/trimmomatic/{sample}_R1_paired.fq.gz",
        forward_unpaired = "reads/trimmomatic/unpaired/{sample}_R1_unpaired.fq.gz",
        reverse_paired = "reads/trimmomatic/{sample}_R2_paired.fq.gz",
        reverse_unpaired = "reads/trimmomatic/unpaired/{sample}_R2_unpaired.fq.gz",
        check = "reads/trimmomatic/out/{sample}_trim_done.txt"
        
    message: "Trimming Illumina adapters from {input.forward_raw} and {input.reverse_raw}"
    shell:'''
        module load java
        java -jar /home/amhorst/programs/Trimmomatic-0.39/trimmomatic-0.39.jar PE -threads 8 -phred33 {input.forward_raw} {input.reverse_raw} \
        {output.forward_paired} {output.forward_unpaired} \
        {output.reverse_paired} {output.reverse_unpaired} \
        ILLUMINACLIP:/home/amhorst/programs/Trimmomatic-0.39/adapters/TruSeq3-PE.fa:2:30:10 \
        SLIDINGWINDOW:4:30 MINLEN:50 && touch {output.check}
        '''
        

## Use Bowtie2 to map back reads to tomato genome
- Mapping to https://www.ncbi.nlm.nih.gov/assembly/GCA_012431665.1#/st (tomato)
- Make a bowtie2 index
- map reads to index using bowtie2
- http://bowtie-bio.sourceforge.net/bowtie2/index.shtml
- to save room we write to --un-conc-gz --> will be fq.gz file with reads not mapped to reference
- the sam file will contain only mapped reads (mapped to host genome) and will be removed as soon script is done


In [ ]:
# create folders in the reads folders
cd 2021_tomato_rhizo/reads
mkdir rmhost
mkdir rmhost/samfiles
mkdir rmhost/unpaired


# Load module
module load bowtie2

# make reference file
cd /home/amhorst/plant_genomes/
bowtie2-build tomato_genome.fa tomato_genome 


In [ ]:
# Check names
for f in *_R1_*.fq.gz; do
echo $f
echo ${f%_R1_*}_rmphix.fq.gz
echo ./unpaired/${f%_R1_*}_R2_unpaired.fq.gz
echo ./unpaired/${f%_R1_*}_R1_unpaired.fq.gz
done

'''PAIRED'''

#!/bin/bash
#SBATCH --job-name=bowtie
#SBATCH --nodes=1
#SBATCH -t 4:00:00 
#SBATCH --ntasks=12
#SBATCH --output=../../out/bowtiehost_%j.out
#SBATCH --error=../../err/bowtiehost_%j.err
#SBATCH --partition=bmm

#load modules
module load bowtie2
aklog

for f in *_R1_*.fq.gz; do
echo $f
bowtie2 --threads 12 --sensitive \
-x /home/amhorst/plant_genomes/tomato_genome \
-1 $f -2 ${f%_R1_*}_R2_rmphix.fq.gz \
--un-conc-gz ../rmhost/${f%%_R1_*} -S ../rmhost/samfiles/${f%%_R1_*}paired.sam --no-unal \
&& rm ../rmhost/samfiles/${f%%_R1_*}paired.sam
done

'''UNPAIRED'''

#!/bin/bash
#SBATCH --job-name=bowtie
#SBATCH --nodes=1
#SBATCH -t 4:00:00 
#SBATCH --ntasks=12
#SBATCH --output=../../out/bowtiehost_%j.out
#SBATCH --error=../../err/bowtiehost_%j.err
#SBATCH --partition=bmm

#load modules
module load bowtie2
aklog

for f in *_R1_*.fq.gz; do
echo $f
bowtie2 --threads 12 --sensitive \
-x /home/amhorst/plant_genomes/tomato_genome \
-U ./unpaired/${f%_R1_*}_R2_unpaired.fq.gz,./unpaired/${f%_R1_*}_R1_unpaired.fq.gz \
--un-gz ../rmhost/unpaired/${f%%_R1_*} -S ../rmhost/samfiles/${f%%_R1_*}unpaired.sam --no-unal \
&& rm ../rmhost/samfiles/${f%%_R1_*}unpaired.sam
done

'''RENAME'''

# Bowtie is done but gives files weird names: Rename

# for the unpaired ones
for f in $(ls *L00*) ; do 
echo $f
echo ${f%%}_unpaired.fq.gz
mv $f ${f%%}_unpaired.fq.gz
done

# for paired
for f in $(ls *.1) ; do 
echo $f
echo ${f%%.1*}_R1_rmhost.fq.gz
mv $f ${f%%.1*}_R1_rmhost.fq.gz
done

for f in $(ls *.2) ; do 
echo $f
echo ${f%%.2*}_R2_rmhost.fq.gz
mv $f ${f%%.2*}_R2_rmhost.fq.gz
done

# Remove the S_ part
for f in *.fq.gz
do 
echo $f
newname=$( echo $f | sed -r 's/_S[0-9]{3}_/_/' ); mv $f $newname; done
done


## Assembly of the clean reads using MEGAHIT
- https://github.com/voutcn/megahit
- https://academic.oup.com/bioinformatics/article/31/10/1674/177884

In [ ]:
# megahit doesnt want to create the temp folder
# this version is currently working
# megahit 
rule PE_assembly:
    input: 
        forward_read = "reads/rmhost/{sample}_R1_rmhost.fq.gz", 
        reverse_read = "reads/rmhost/{sample}_R2_rmhost.fq.gz",
        unpaired = "reads/rmhost/unpaired/{sample}_unpaired.fq.gz"
    output:
        check = "assemblies/out/{sample}_assem_done.txt",
        out_contig = "assemblies/megahit_final/{sample}.contigs.fa" 
    params:
        output_folder = "assemblies/megahit_final",
        output_temp = "assemblies/megahit_temp"
    message: "paired end assembly on {input.forward_read}"
    shell:'''
    mkdir -p  assemblies/megahit_temp/
    module load megahit
    
    # megahit does not allow force overwrite, so each assembly needs to take place in it's own directory. 
    megahit -1 {input.forward_read} -2 {input.reverse_read} \
    -r {input.unpaired} \
    -t 16 --continue --k-min 27 --min-contig-len 1000 --presets meta-large -m 0.095 \
    --out-dir {params.output_temp}/{wildcards.sample} \
    --out-prefix {wildcards.sample} && \
    mv {params.output_temp}/{wildcards.sample}/{wildcards.sample}.contigs.fa \
    {params.output_folder} && touch {output.check}
    '''

## Renaming contigs and split on lenght
- MEGAHIT outputs contigs with spaces in filenames, don't like that
- Only keep contigs > 10kb for predicting viruses
- Using bbmap package
- https://jgi.doe.gov/data-and-tools/software-tools/bbtools/
- https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0185056

In [ ]:
# rename contigs and  split off contigs >10kb
# rename contigs and  split off contigs >10kb
rule rename_contigs:
    input:
        contigs = "assemblies/megahit_final/{sample}.contigs.fa",
        check = "assemblies/out/{sample}_assem_done.txt"
    output:
        contigs_renamed = "outputs/megahit_final/{sample}.contigs_renamed.fa",
        assembly_stats = "outputs/megahit_final/stats_assembly/{sample}.megahit.stats.txt",
        contigs_10kb = "outputs/megahit_final/{sample}.contigs_10kb.fa",
        check = "assemblies/out/{sample}_reformat_done.txt"
    # print a message on what its doing
    message: "renaming contigs of sample {input.contigs}"
    shell:'''
    module load bbmap
    rename.sh in={input.contigs} out={output.contigs_renamed} prefix={wildcards.sample} && \
    reformat.sh in={output.contigs_renamed} out={output.contigs_10kb} minlength=10000 &&
    stats.sh in={output.contigs_renamed} gc={output.assembly_stats} gcformat=3 && touch {output.check}
    '''

## Use VIBRANT to predict viral contigs from all contigs
- https://github.com/AnantharamanLab/VIBRANT
- https://pubmed.ncbi.nlm.nih.gov/32522236/

In [ ]:
# Same thing with a temp folder for snakemake as with MEGAHIT
# run vibrant
rule vibrant:
    input:
        contigs = "outputs/megahit_final/{sample}.contigs_10kb.fa",
        check = "assemblies/out/{sample}_reformat_done.txt"
    output:
        vibrant_contig = "outputs/vibrant_final/{sample}.contigs_10kb.phages_combined.fna",
        check = "outputs/out/{sample}_vibrant_done.txt"
    params:
        output_folder = "outputs/vibrant_final",
        output_temp = "outputs/vibrant_temp"
    # print a message on what its doing
    message: "running vibrant on {input.contigs}"
    shell:'''
    mkdir -p  outputs/vibrant_temp/
    
    set +u
    source ~/.bashrc
    conda activate vibrant
    set -u
    
    VIBRANT_run.py -i {input.contigs} -folder {params.output_temp}/{wildcards.sample} \
    -t 8 -l 10000 -o 4 -virome && \
    mv {params.output_temp}/{wildcards.sample}/VIBRANT_{wildcards.sample}.contigs_10kb/VIBRANT_phages_{wildcards.sample}.contigs_10kb/{wildcards.sample}.contigs_10kb.phages_combined.fna \
    {params.output_folder} && touch {output.check} 
    '''

## Use dRep to dereplicate viral contigs. 
- https://drep.readthedocs.io/en/latest/
- https://www.nature.com/ismej/articles

In [ ]:
# split contigs into individual fastas (dRep accepts individual contigs only)
mkdir contigs
cd ./contigs
awk '/^>/ {OUT=substr($0,2) ".fa"}; OUT {print >OUT}' ../*.fa 


# Activate dRep conda instance
source ~/.bashrc
conda activate drep

# Load other programs that dRep needs
module load mummer
module load mash
module load bowtie2

# Run dRep at 95% ANI over 85% of length of longest contigs
dRep dereplicate ./drep --S_algorithm ANImf --ignoreGenomeQuality -l 10000 -sa 0.95 -nc 0.85 \
-g ./contigs/*.fa -p 14


## Use Bowtie2 to map back reads to viral contigs
- Make a bowtie2 index
- map reads to index using bowtie2
- http://bowtie-bio.sourceforge.net/bowtie2/index.shtml
- https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3322381/
- Convert samfiles to bamfiles using samtools
- index bamfiles using samtools
- http://www.htslib.org/
- https://academic.oup.com/bioinformatics/article/25/16/2078/204688


In [ ]:
# map all clean reads both to final set of vOTUs

#load modules
module load bowtie2

bowtie2 --threads 12 --sensitive \
-x 220118_tomato_rhizo_finaldrep \
-1 $1 \
-2 ${1%_R1*}_R2_rmhost.fq.gz \
-S ./2021_tomato_rhizo/mapping/final/${1%%_R1_*} \
--no-unal --sensitive 

## Use samtools to convert 
- Convert sam to bam files
- Index the bam files
- https://github.com/samtools/samtools

In [ ]:
# sam to bam 
for f in *.sam
do
samtools view -@ 12 -F 4 -bS $f | samtools sort > ${f%.sam*}.bam
done

# index the bam files
for f in *.bam
do
samtools index $f
done



## Use CoverM to make a coverage table
- https://github.com/wwood/CoverM
- https://wwood.github.io/CoverM/coverm-genome.html#author

In [ ]:
# Create coverage table
source ~/.bashrc
conda activate coverm
    
coverm contig -m mean --min-covered-fraction 0.75 -b *.bam > 220118_coverM_75_tomato.tsv
